In [1]:
import pandas as pd
import numpy as np
import spacy
import mysql.connector

In [2]:
nlp = spacy.load('en_core_web_sm')

## Load Data 

In [3]:
chunksize = 1000
for chunk in pd.read_csv('./actor_action_desc.csv', chunksize=chunksize):
    print(chunk.head())
    break

           0                  1  \
0  Plaintiff  Application filed   
1  Plaintiff   Appearance filed   
2  Plaintiff  Application filed   
3      Court    Event scheduled   
4      Court  Return of Service   

                                                   2  
0  Application filed on 04/27/2011 at Boston Muni...  
1  Appearance for Potfolio Recovery Associates, L...  
2  Supplementary process application filed; filin...  
3         HEARING SCHEDULED for 06/09/2011 10:00 AM.  
4  Return of service on complaint and summons to ...  


In [4]:
temp = chunk.iloc[:10].values
for description in temp[:,2]:
    doc = nlp(description)
    for i,token in enumerate(doc):
        print(token.text, token.tag_, token.head.text, token.dep_)
    print('\n\n')
        

Application NN filed nsubj
filed VBD filed ROOT
on IN filed prep
04/27/2011 CD on pobj
at IN filed prep
Boston NNP Court compound
Municipal NNP Court compound
Court NNP at pobj
, , Court punct
Dorchester NNP Div compound
Div NNP Court appos
.. . filed punct



Appearance NNP Appearance ROOT
for IN Appearance prep
Potfolio NNP Associates compound
Recovery NNP Associates compound
Associates NNPS for pobj
, , Appearance punct
Llc NNP filed nsubj
filed VBN Appearance acl
by IN filed agent
Attorney NNP STEPHEN compound
STEPHEN NNP by pobj
A. NNP Appearance punct
WIENER NNP MA nmod
340 CD WIENER nummod
Main NNP Suite compound
Street NNP Suite compound
Suite NNP WIENER appos
959 CD Suite nummod
Worcester NNP MA compound
MA NN MA ROOT
01608 CD 0613 compound
- HYPH 0613 punct
0613 CD MA appos
BBO NNP MA appos
# $ MA appos
656119 CD MA punct



Supplementary JJ process amod
process NN application compound
application NN filed nsubj
filed VBD filed ROOT
; : filed punct
filing NN fee compound
fee 

In [53]:
mydb = mysql.connector.connect(host='73.38.248.152', user='buspark', password='U@5p1r3!')

if (mydb):
    print("Connection Successful")

else:
    print("Connection Unsuccessful")

mycursor = mydb.cursor()

mycursor.execute("Show databases")

for db in mycursor:
    print(db)

Connection Successful
('information_schema',)
('civica_courtdocs',)
('wp_courtdocs',)
('wp_courtdocs_NORMALIZED',)


In [56]:
action_null = pd.read_sql("SELECT * FROM wp_courtdocs.cdocs_case_action_index as c_a_index \
                          WHERE c_a_index.action = ' ' and c_a_index.actor = ' ' LIMIT 0, 1000;"
                          ,con = mydb)
action_null.head()

,case_action_id,case_id,actor,action,description,date_time,file_reference_number,last_indexed
0,12248,12238,,,NOTICE: Citation on Petition for Formal Adjud...,2014-09-30,0,2017-02-05 14:49:43
1,12250,12238,,,Citation Filed; Served as Ordered,2014-10-31,0,2017-02-05 14:49:43
2,12251,12238,,,"An Interested Person,Johanna Soris, Asst. Atto...",2014-10-31,0,2017-02-05 14:49:43
3,12254,12238,,,Letters of Authority for Personal Representative,2014-11-06,0,2017-02-05 14:49:43
4,12308,12304,,,NOTICE: Citation on Petition for Formal Adjud...,2014-07-02,0,2017-02-05 14:49:43


In [58]:
action_null['description'].tolist()

['NOTICE:  Citation on Petition for Formal Adjudication Issued.',
 'Citation Filed; Served as Ordered',
 'An Interested Person,Johanna Soris, Asst. Attorney General , Filed Form MPC 455 toPetition for Formal Adjudication and for the appointment of PR',
 'Letters of Authority for Personal Representative',
 'NOTICE:  Citation on Petition for Formal Adjudication Issued.',
 'Suspicious Death Affidavit',
 'Citation Filed; Served as Ordered',
 'Letters of Authority for Personal Representative',
 'Assent of Mary Jeanne Mullen',
 'It is ordered that notice by made by Publication, Mail, In hand service.  If by publication, published in The Recorder, Greenfield.\n\n',
 'Citation Filed; Served as Ordered',
 'Decree Appointing Fiduciary dated 12/29/10',
 'Inventory Issued',
 'Inventory  Filed',
 'Subsequent Action First and Final Account filed',
 'Appearance by attorney Deborah T Jankowski Esq.\n\n',
 'Judgment on First and Final Account',
 "An Interested Person,Curtis Janey, o/b/o Isaiah C. Janey

## Semantic Role Labelling

In [2]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging


In [3]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")

error loading _jsonnet (this is expected on Windows), treating C:\Users\dell\AppData\Local\Temp\tmpbt8nfpe9\config.json as plain json


In [5]:
example_1 = 'Massachusetts Estate Tax Closing Letter'
example_2 = "Motion for appointment of special process server filed by 1835 Middlesex Street Condominium Trust by its Board of Trustees."
example_3 = "Inventory filed"
example_4 = "Attorney fees waived by plaintiff(s)."

In [10]:
sentence = example_4
pred = predictor.predict(sentence)
words = pred['words']
for obj in pred['verbs']:
    v = obj['verb']
    tags = obj['tags']
    arg0 = ''
    arg1 = ''
    if "B-ARG0" in tags:
        start = tags.index("B-ARG0")
        end = max([i for i, x in enumerate(tags) if x == "I-ARG0"] + [start]) + 1
        arg0 = words[start:end]

    if "B-ARG1" in tags:
        start = tags.index("B-ARG1")
        end = max([i for i, x in enumerate(tags) if x == "I-ARG1"] + [start]) + 1
        arg1 = words[start:end]
        
    print('subject:',arg0)
    print('v:',v)
    print('object:',arg1)
    print('\n')

subject: ['by', 'plaintiff(s']
v: waived
object: ['Attorney', 'fees']


